In [4]:
import os
import cv2
import numpy as np
import json
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
DATA_PATH = os.path.join(os.getcwd(), '../data/6_pts')
print(DATA_PATH)
PROCESSED_DATA_PATH = os.path.join(os.getcwd(), '../processed/6_pts')
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)
if not os.path.exists(os.path.join(PROCESSED_DATA_PATH,'images')):
    os.makedirs(os.path.join(PROCESSED_DATA_PATH,'images'))

/home/boomalope/nerf_estimator/src/nerf_estimator/nerf_vision/scripts/../data/6_pts


In [6]:
#Gazebo Param: 
#Definition: https://github.com/ros-simulation/gazebo_ros_pkgs/blob/398cde933cba5d72921055d57d4faa47c4445654/gazebo_plugins/src/gazebo_ros_camera_utils.cpp#L524

In [7]:
#Camera parameters
#Definitions: https://docs.nerf.studio/quickstart/data_conventions.html

fl_x = 762.72 # focal length in x
fl_y = 762.72# focal length in y
w = 1280 # image width
h = 720 # image height
cx = w//2 # optical center in x
cy = h//2 # optical center in y
camera_model = "OPENCV" # camera model
k1 = 0 # radial distortion coefficient k1
k2 = 0 # radial distortion coefficient k2
k3 = 0 # radial distortion coefficient k3
p1 = 0 # tangential distortion coefficient p1
p2 = 0 # tangential distortion coefficient p2

In [8]:
def from_tf_to_matrix(tf):
    #tf is the frame of the robot relative to the camera
    xyz = tf[:3]
    q = tf[3:7]
    goal = tf[7:]

    #Rotation matrix
    r = Rotation.from_quat(q)
    r = r.as_matrix()

    #Additional rotation  matrix to shift camera from +x to -z direction and make the x axis point to the right
    A = Rotation.from_euler('YZX', [-90, -90, 0], degrees=True).as_matrix()

    #Translation matrix in the world frame
    t = np.zeros((4,4))
    t[:3,:3] = np.matmul(r.T,A)
    t[:3,3] = -np.matmul(r.T,xyz)
    t[3,3] = 1

    print(t[:3,3])
    print(Rotation.from_matrix(t[:3,:3]).as_euler('xyz', degrees=True))
    
    return t

In [9]:
data_cnt = 0
frames = []
while True:
    #Get data paths
    data_img = os.path.join(DATA_PATH, '{}.png'.format(data_cnt))
    data_tf= os.path.join(DATA_PATH, '{}.txt'.format(data_cnt))

    if not os.path.exists(data_img):
        break
    
    #Get tf as np array
    tf = np.loadtxt(data_tf, delimiter=',')


    file_path = "images/{}.png".format(data_cnt)
    transform_matrix = from_tf_to_matrix(tf)

    #Append to frames
    frames.append({
        "file_path": file_path,
        "transform_matrix": transform_matrix.tolist(),
    })

    #Save image in processed folder
    img = cv2.imread(data_img)
    #Change img from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(PROCESSED_DATA_PATH, file_path), img)
    
    data_cnt += 1       

[-2.40289595e+00 -6.15973369e-01  2.58897747e-04]
[ 8.99902097e+01  2.21348801e-03 -9.68744050e+01]
[-2.40289190e+00 -6.17115596e-01  5.09322913e-05]
[ 8.99957806e+01  1.14097867e-03 -9.68473837e+01]
[-2.43439072e+00 -4.78952883e-01  4.56113753e-05]
[ 8.99950091e+01  3.46372301e-03 -1.00119560e+02]
[-2.48030367e+00 -6.62925627e-02  1.70010768e-04]
[ 8.99936871e+01 -1.06243407e-03 -1.09718901e+02]
[-2.45804013e+00  3.39521318e-01  1.87215760e-04]
[ 8.99935925e+01 -1.91452724e-03 -1.19114626e+02]
[-2.36916656e+00  7.39215950e-01  1.01393643e-04]
[ 8.99952353e+01 -6.70622121e-04 -1.28581330e+02]
[-2.24615927e+00  1.05559699e+00  1.28282721e-04]
[ 8.99937987e+01  1.31041140e-03 -1.36423924e+02]
[-2.05930336e+00  1.38430693e+00  2.13308670e-04]
[ 8.99925602e+01 -9.21547224e-04 -1.45155390e+02]
[-1.82147932e+00  1.68546176e+00  1.74002270e-04]
[ 8.99938820e+01 -1.81612233e-03 -1.54030779e+02]
[-1.51924998e+00  1.96257950e+00  1.53517549e-04]
[ 8.99934333e+01  6.41811062e-04 -1.63515213e+02]


[-4.16232445e-01  2.44661451e+00  6.12584709e-05]
[ 8.99985604e+01 -6.66422234e-03  1.68408694e+02]
[-3.06338460e-02  2.48159114e+00 -1.35076194e-04]
[ 9.00022603e+01 -3.67158041e-03  1.59456268e+02]
[3.82122635e-01 2.45214592e+00 3.09776409e-04]
[ 8.99934489e+01 -9.34771366e-03  1.49880866e+02]
[6.87750439e-01 2.38452151e+00 1.95403192e-05]
[ 8.99987979e+01 -4.61621267e-03  1.42646725e+02]
[1.11134509e+00 2.21922581e+00 1.09642473e-04]
[ 8.99976322e+01 -7.35696769e-03  1.32147275e+02]
[1.41258968e+00 2.04060178e+00 5.33165923e-05]
[ 8.99992182e+01 -7.84727314e-03  1.24051313e+02]
[ 1.67860836e+00  1.82766214e+00 -2.23309068e-04]
[ 9.00047847e+01 -4.53953700e-03  1.16170334e+02]
[ 1.95783175e+00  1.52435917e+00 -2.35780413e-05]
[ 9.00013898e+01 -8.53058108e-03  1.06634729e+02]
[ 2.21001862e+00  1.12810077e+00 -9.60619719e-05]
[ 9.00032896e+01 -8.79703204e-03  9.57721132e+01]
[ 2.34340841e+00  8.17582135e-01 -1.86095536e-04]
[ 9.00055161e+01 -8.78865051e-03  8.79713372e+01]
[ 2.43759414

In [10]:
transform_dictionary = {}
transform_dictionary["fl_x"] = fl_x
transform_dictionary["fl_y"] = fl_y
transform_dictionary["w"] = w
transform_dictionary["h"] = h
transform_dictionary["cx"] = cx
transform_dictionary["cy"] = cy
transform_dictionary["k1"] = k1
transform_dictionary["k2"] = k2
transform_dictionary["k3"] = k3
transform_dictionary["p1"] = p1
transform_dictionary["p2"] = p2
transform_dictionary["camera_model"] = camera_model
transform_dictionary["frames"] = frames

#Save as json 
with open(os.path.join(PROCESSED_DATA_PATH, "transforms.json"), 'w') as outfile:
    json.dump(transform_dictionary, outfile,indent=4)
